In [ ]:
# This cell is tagged with "parameters" and used by papermill

# default settings
inpath = "" # type: str
outpath = "" # type: str
edge_element = "" # type: str
normalise = True # type: bool
auto_processing = True # type: bool

## Usage
This notebook takes a scan file `inpath` and a line group `edge_element`, performs windowing of this line group, and saves a two-column text file `outpath`: the first column is the energy in keV, and the second column is the summed windowed MCA intensity across the 4 channels.

### Parameters
`inpath` : str      
    The full path of the scan file, e.g. `"/dls/i14/data/2024/mg38234-1/scan/i14-340327.nxs"`     
`outpath` : str     
    The summed intensity as a two-column text file, e.g. `"/dls/i14/data/2024/mg38234-1/processed/xanes_point/i14-340327_xanes_point.txt"`     
`edge_element` : str    
    The line group of interested, e.g. `"Fe-Ka"`     
`normalise` : bool       
    Whether to normalise the windowed intensity by `adc_ionCh1a` etc. Default to `True`.      
`auto_processing` : bool      
    The flag to state this is an auto- or post-processing. Default to `True`.

**The parameters should be provided by explicitly modifying the top cell content or using tools such as [papermill](https://papermill.readthedocs.io/en/latest/index.html). If the notebook is run as is, please define the parameters accordingly.**

### Dependencies
The windowing is carried by [i14-utility-xanes](https://gitlab.diamond.ac.uk/i14/i14_utility).

- matplotlib
- numpy
- h5py
- i14-utility (https://gitlab.diamond.ac.uk/i14/i14_utility)

In [ ]:
from pathlib import Path
import re

import numpy as np
import h5py
import matplotlib.pyplot as plt

from i14_utility.xanes.window_xrf import channel_start_end, window_mca

### Validation

In [ ]:
if not Path(inpath).is_file():
    msg = "Check the provided NeXus file path."
    raise FileNotFoundError(msg)

### Override the path of output file
Save the text and png files inside `xanes_point`.

In [ ]:
output_folder = Path(outpath).parent / "xanes_point"
output_folder.mkdir(parents=True, exist_ok=True)

print(f"output_folder is {output_folder}")

In [ ]:
scan_num = re.search(r"^.*i14-(\d+)\.nxs$", inpath).group(1)

txt_outpath = str(output_folder / f"i14-{scan_num}_xanes_point.txt")
png_outpath = str(output_folder / f"i14-{scan_num}_xanes_point.png")

print(f"The txt file will be saved as {txt_outpath}")
print(f"The PNG file will be saved as {png_outpath}")

### Load the dataset
Check if I0 can be found as in rare cases only some of the I0 can be found, hence the for-loop check. 

Normalisation will be skipped for incomplete set of I0 despite `normalise` is `True`.

In [ ]:
I0_total = 0
found_I0 = [False, False, False, False]
XSP3_KEY = "/entry1/xsp3_addetector/"

with h5py.File(inpath, "r") as f:
    if XSP3_KEY not in f:
        msg = "Missing the group {XSP3_KEY} in this point XANES dataset."
        raise RuntimeError(msg)
    
    mca = f[f"{XSP3_KEY}/data"][()]
    energy = f[f"{XSP3_KEY}/dcm_enrg"][()]

    for k, ch_id in enumerate(("1a", "1b", "2a", "2b")):
        if f"adc_ionCh{ch_id}" in f[XSP3_KEY]:
            found_I0[k] = True
            I0_total += f[f"{XSP3_KEY}/adc_ionCh{ch_id}"][()]
            print(f"{XSP3_KEY}/adc_ionCh{ch_id} found.")
        else:
            print(f"{XSP3_KEY}/adc_ionCh{ch_id} NOT found.")

### Determine the channels for windowing
`width` is set to 20, keep the same behaviour of the old notebook. These data are probably sharp so the width could be smaller (instead of 40).

In [ ]:
start_ch, end_ch = channel_start_end(edge_element, width=20)

### Window the MCA data by the provided line group
`scan_shape` is the same as the number of energy; `scan_model` is irrelevant here so we just pass it as an empty string.

In [ ]:
scan_shape = (energy.size, )
scan_model = ""

windowed = window_mca(np.squeeze(mca), start_ch, end_ch, scan_shape, scan_model)

In [ ]:
print("Windowed data shape", windowed.shape)
print("Energy data shape", energy.shape)

### Perform normalisation
Skip normalisation if not all the I0 are found even the `normalise` flag is `True`.

In [ ]:
if to_normalise := (normalise and all(found_I0)):
    print('Normalisation is performed')
    I0_max = I0_total.max()
    I0_frac = [i0 / I0_max for i0 in I0_total]
    
    windowed_normalised = (windowed / I0_frac).astype(windowed.dtype)
else:
    print('Normalisation is skipped.')

### Plot the windowed data and save it as PNG

In [ ]:
fig, ax = plt.subplots()
ax.plot(energy, windowed, "b-", label="summed")

if to_normalise:
    ax.plot(energy, windowed_normalised, "r-", label="normalised")

ax.set_xlabel("Energy (keV)")
ax.set_ylabel("Summed intensity")
ax.legend()
fig.tight_layout()
plt.savefig(png_outpath)

### Save the data as txt file

In [ ]:
if to_normalise:
    np.savetxt(txt_outpath, np.column_stack((energy*1000, windowed_normalised)))
else:
    np.savetxt(txt_outpath, np.column_stack((energy*1000, windowed)))